# Recommendation System

## Задание 1

Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов 
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). Оставь в датасете только фильмы, которые вышли в "релиз".\
Выведи количество фильмов, оставшихся после фильтрации.

In [1]:
import pandas as pd

#Загрузка данных о фильмах
credits_data = pd.read_csv('../datasets/tmdb_5000_credits.csv')
movies_data = pd.read_csv('../datasets/tmdb_5000_movies.csv')

#Объединение данных о фильмах и касте фильмов
dataset = pd.merge(movies_data, credits_data, left_on='id', right_on='movie_id')

#Фильтрация фильмов, которые вышли в "релиз"
released_movies = dataset[dataset['status'] == 'Released']

#Вывод количества оставшихся фильмов
num_movies = len(released_movies)
print(f"Количество фильмов после фильтрации: {num_movies}")


Количество фильмов после фильтрации: 4795


## Задание 2

Самый наивный подход к рекомендации фильмов - рекомендовать фильмы с лучшими оценками пользователей. Фильмы, которые пользуются большей популярностью и признанием критиков, с большей вероятностью понравятся среднему зрителю.

Для справедливой оценки фильмов возьмем текущую рейтинговую систему IMDB (weighted rating (WR)), которая рассчитывается по формуле:
$$WR = \frac{v}{v + m} ⋅ R + \frac{m}{v + m} ⋅ C$$ 
$v$ - количество голосов \
$m$ - количество голосов для включения в финальную таблицу \
$R$ - средняя оценка \
$C$ - средняя оценка всех фильмов 

Имплементируй функцию `weighted_rating`. С её помощью расcчитай рейтинг для каждого фильма и сохрани его в колонку `simple_score`.\
Выведи топ-5 фильмов по получившемуся рейтингу.
> В качестве параметра $m$ выбери 95-й квантиль количества голосов.

In [2]:
def weighted_rating():
    pass 

In [3]:
#Импорт необходимых библиотек
import numpy as np

#Реализация функции weighted_rating
def weighted_rating(x, m, C):
    vote_count = x['vote_count']
    vote_average = x['vote_average']
    return (vote_count / (vote_count + m) * vote_average) + (m / (m + vote_count) * C)

#Рассчет рейтинга для каждого фильма
m = released_movies['vote_count'].quantile(0.95)
C = released_movies['vote_average'].mean()
released_movies['simple_score'] = released_movies.apply(lambda x: weighted_rating(x, m, C), axis=1)

#Вывод топ-5 фильмов по рейтингу
top_movies = released_movies.nlargest(5, 'simple_score')
print(top_movies[['original_title', 'simple_score']])

                original_title  simple_score
1881  The Shawshank Redemption      7.849025
65             The Dark Knight      7.773990
662                 Fight Club      7.761012
96                   Inception      7.736496
3232              Pulp Fiction      7.714726


C:\Users\evpet\AppData\Local\Temp\ipykernel_18392\1998319820.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  released_movies['simple_score'] = released_movies.apply(lambda x: weighted_rating(x, m, C), axis=1)


## Задание 3

Такой подход к рекомендациям очень наивен, так как не учитывает информацию о самом фильме (жанр, режиссер, описание, актеры и т.п). \
**Content Based Filtering** (Фильтрация на основе содержания) - тип рекомендательной системы, которая предлагает пользователям похожие элементы на основе конкретного элемента. Общая идея этих рекомендательных систем заключается в том, что если человеку понравился определенный товар, то ему понравится и похожий на него товар.

<center><img src="../misc/images/content.png" alt= “” width="300" height="500">

Реализуем алгоритм рекомендации на основе описания фильма. Для это требуется провести предобработку текста:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Расcчитай [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильма

Выведи размер получившейся матрицы Tf-Idf

> Для [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) используйте параметры по умолчанию 

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Замена NaN в описании фильма на пустой символ ''
released_movies.loc[:, 'overview'] = released_movies.loc[:, 'overview'].fillna('')

#Создание объекта TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')

#Расчет Tf-Idf для описания фильма
tfidf_matrix = tfidf.fit_transform(released_movies['overview'])

#Вывод размера матрицы Tf-Idf
print(f"Размер матрицы Tf-Idf: {tfidf_matrix.shape}")

Размер матрицы Tf-Idf: (4795, 20970)


## Задание 4

Теперь тебе необходимо вычислить показатель сходства между описаниями фильмов. Используем косинусное расстояние, оно рассчитывается по формуле:
$$cos(Θ) = \frac{A ⋅ B}{∥A∥ ∥B∥} = \frac{ Σ_{i=1}^{n} A_i ⋅ B_i } { \sqrt{Σ_{i=1}^{n}A_{i}^{2}} ⋅ {\sqrt{Σ_{i=1}^{n}B_{i}^{2}}}}$$
Но поскольку мы использовали векторизатор TF-IDF на предыдущем шаге, достаточно вычислить скалярное произведение, которое и даст оценку косинусного сходства. Рассчитать его можно через [linear_kernel](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.linear_kernel.html). Результат сохрани в переменную `cosine_sim`.

Выведи размер получившейся матрицы `cosine_sim`.

In [5]:
from sklearn.metrics.pairwise import linear_kernel

#Рассчет скалярного произведения для расчета показателя сходства
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#Вывод размера матрицы cosine_sim
print(f"Размер матрицы cosine_sim: {cosine_sim.shape}")



Размер матрицы cosine_sim: (4795, 4795)


## Задание 5

Напиши функцию `get_recommendations`. На вход она принимает:
* `movies_dataset` - датасет фильмов
* `title` - название фильма, для которого мы будем искать похожие
* `cosine_sim` - матрица расстояний между описаниями
* `top_k` - топ-k cхожих фильмов

Возвращает top_k названий фильмов, описание которых похоже на выбранный фильм.\
Выведи топ-5 фильмов для `title='Saving Private Ryan'`

In [6]:
def get_recommendations(movies_dataset, title, cosine_sim, top_k=10):
    pass 

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_recommendations(movies_dataset, title, cosine_sim, top_k=10):
    # Получение индекса фильма по названию
    idx = movies_dataset[movies_dataset['title'] == title].index[0]

    # Получение оценок похожести фильмов на выбранный фильм
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Сортировка фильмов по оценкам похожести в убывающем порядке
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Получение индексов топ-K похожих фильмов
    top_indexes = [i[0] for i in sim_scores[1:top_k+1]]

    # Получение названий топ-K похожих фильмов
    top_movies = movies_dataset.iloc[top_indexes]['title']

    return top_movies

# Загрузка датасета
movies_dataset = pd.read_csv('../datasets/tmdb_5000_movies.csv')


# Создание матрицы расстояний
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_dataset['overview'].fillna(''))
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Вызов функции и вывод результатов
title = 'Saving Private Ryan'
top_movies = get_recommendations(movies_dataset, title, cosine_sim, top_k=5)
print(top_movies)

787        The Great Raid
586     The Monuments Men
290     The Expendables 2
4168            Abandoned
3516            The Train
Name: title, dtype: object


## Задание 6

Еще один подход к построению рекомендательной системы - подход на основе сходства между пользователями. Этот подход называется **Collaborative Filtering** (Коллаборативная фильтрация).
<center><img src="../misc/images/all.png" alt= “” width="600" height="700"></center>
Коллаборативная фильтрация - это тип рекомендательной системы, которая использует поведение и предпочтения похожих пользователей для рекомендации товаров или продуктов конкретному пользователю. Система собирает данные о прошлом поведении пользователей, такие как покупки, рейтинги и отзывы, и анализирует их для выявления закономерностей и сходства между пользователями. На основе этих закономерностей система рекомендует товары, которые понравились или были приобретены другими такими же пользователями в прошлом.

Для реализации Коллаборативной фильтрации нам потребуются оценки пользователей [ratings](../datasets/ratings.csv).

>userId - id пользователя \
movieId - id фильма \
rating - оценка фильма (от 0 до 5)\
timestamp - время оценки


Воспользуйся библиотекой [surprise](https://surpriselib.com/) для обучения модели оценки рейтинга фильма [SVD](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD). Выведи средние значения 'RMSE', 'MAE' на кросс-валидации с параметрами `cv=5`.

In [8]:
from surprise import SVD, Reader, Dataset
from surprise.model_selection import cross_validate

#Загрузка данных о рейтингах
ratings_df = pd.read_csv('../datasets/ratings.csv')

#Создание объекта Reader для Surprise
reader = Reader(rating_scale=(0, 5))

#Загрузка данных о рейтингах в объект Dataset для Surprise
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

#Создание модели SVD
model = SVD()

#Проведение перекрестной проверки
cv_results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

#Вывод среднего значения RMSE и MAE
print("Среднее значение RMSE:", round(cv_results['test_rmse'].mean(), 3))
print("Среднее значение MAE:", round(cv_results['test_mae'].mean(), 3))

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9014  0.9011  0.8955  0.8946  0.8954  0.8976  0.0030  
MAE (testset)     0.6914  0.6940  0.6923  0.6875  0.6902  0.6911  0.0022  
Fit time          8.95    7.40    8.13    7.73    7.85    8.01    0.52    
Test time         1.48    1.11    1.11    1.42    1.27    1.28    0.15    
Среднее значение RMSE: 0.898
Среднее значение MAE: 0.691
